# Introduction to Database Design
---------------------------------------------------------------------------------------------------------

In our previous session on databases, we introduced some of the fundamental concepts and definitions applicable to databases in general, along with a brief intro to SQL and SQLite in particular. Some use cases and platforms were also discussed.

In this session, we are going to dig a little deeper into databases as representions of systems and processes. A database with a single table may not feel or function much differently from a spreadsheet. Much of the benefit of using databases results from designing them as models of complex systems in ways that spreadsheets just can't do:

* Inventory control and billing
* Human resources
* Blogs
* Ecosystems

For this session we are also going to play as we go, so let's begin by installing and importing the SQLAlchemy library. This provides an interface to Python's sqlite3 module.

### Install and load SQLAlchemy

In [2]:
#!pip3 install sqlalchemy
#!pip install sqlalchemy

In [3]:
import sqlalchemy

In [4]:
# Some info links:

# https://www.datanamic.com/support/lt-dez005-introduction-db-modeling.html
# https://www.datanamic.com/support/database-normalization.html
# https://www.fullstackpython.com/databases.html
# https://www.sqlalchemy.org/
# https://opentextbc.ca/dbdesign01/chapter/chapter-8-entity-relationship-model/
# https://www.tutorialspoint.com/dbms/er_model_basic_concepts.htm

# Use field notes for examples
# https://www.biodiversitylibrary.org/
# https://www.biodiversitylibrary.org/bibliography/146255#/summary
# https://www.biodiversitylibrary.org/item/246338#page/1/mode/1up

In [5]:
from sqlalchemy import create_engine

In [6]:
engine = create_engine('sqlite:///:memory:', echo=True)

# The Entity Relationship Data Model
------------------------------------------------------------------------------------------------

The entity relationship (ER) model is commonly used to define and develop databases. In the simplest terms, the model defines the things (entities) that are important or interesting within a system or process and the relationships between them.

For demonstration purposes, we will construct an ER model of data recorded in the _ATF observation check-lists October - December 1964 (ATF 6)_, published online by the [Biodiversity Heritage Library](https://www.biodiversitylibrary.org/).

> National Museum of Natural History (U.S.) Pacific Ocean Biological Survey Program (1964). ATF observation check-lists October - December 1964 (ATF 6). https://www.biodiversitylibrary.org/item/246338. DOI: 10.5962/bhl.title.146255 

Looking at the lists at [https://www.biodiversitylibrary.org/item/246338#page/1/mode/1up](https://www.biodiversitylibrary.org/item/246338#page/1/mode/1up), what are some of the challenges related to transferring these data to a flat spreadsheet?


## Entities

Entities are _nouns_, and can be physical or logical:

* People - teachers, students, courses
* Places - stores, websites, states
* Things - donuts, grades, purchases

Entities are represented as tables within a database. 

### Attributes

Entities have properties or attributes which describe them. For each attribute there is domain, or a range of legal values. Domains can be limited by data type - integer, string, etc. - and may be further limited by allowable values. For example, the domain of month names is limited to January, February, etc.

There are several types of attributes:

* __Simple attributes__ are atomic values which cannot be decomposed or divided. Examples include _age_, _last name_, _glaze_, etc.
* __Composite attributes__ consist of multiple simple attributes, such as _address_, _full name_, etc.
* __Multivalued attributes__ can include a set of more than one value. _Phone numbers_, _certifications_, etc. are examples of multivalued attributes.
* __Derived attributes__ can be calculated using other attributes. A common example is _age_, which can be calculated from a date of birth.

![Entity Relationship example diagram]('./images/539px-ER_Diagram_MMORPG.png')

By <a href="https://en.wikipedia.org/wiki/User:TheMattrix" class="extiw" title="en:User:TheMattrix">TheMattrix</a> at the <a href="https://en.wikipedia.org/wiki/" class="extiw" title="w:">English language Wikipedia</a>, <a href="http://creativecommons.org/licenses/by-sa/3.0/" title="Creative Commons Attribution-Share Alike 3.0">CC BY-SA 3.0</a>, <a href="https://commons.wikimedia.org/w/index.php?curid=2278339">Link</a>

### Keys

A key is an attribute or combination of attributes which can be used to uniquely identify individual entities within the entity set. That is, keys enforce a uniqueness constraint.

There are multiple types of keys. 

* A __candidate key__ is a simple or composite key that is both unique and minimal. _Minimal_ here means that every included attribute is needed to establish uniqueness. A table or entity set may have more than one candidate keys.
* A __composite key__ is a key composed of two or more attributes. Composite keys are also minimal.
* A __primary key__ is the candidate key which is selected to uniquely identify entities in the entity set.
* A __foreign key__ is an attribute the references the primary key of another table or entity set in the database.

#### Exercise

Use the **WWW SQL Designer** at [http://ondras.zarovi.cz/sql/demo/](http://ondras.zarovi.cz/sql/demo/)

In [ ]:
# http://ondras.zarovi.cz/sql/demo/